## 문장 생성 모델

### 각종 설정

- 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 65.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7ec0e54a7f427d9be1c98433e13f54a4e4083011bbcaea81276e684b65ee25a2
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

#### 모델 환경 설정

- pretrained_model_name: 프리트레인 마친 언어모델의 이름
  * (단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 함)
  * 이번 튜토리얼에서는 SK텔레콤이 공개한 KoGPT2(skt/kogpt2-base-v2)를 사용

- downstream_corpus_name: 다운스트림 데이터의 이름

- downstream_model_dir: 파인튜닝된 모델의 체크포인트가 저장될 위치. 
  * /gdrive/My Drive/nlpbook/checkpoint-generation: 자신의 구글 드라이브의 내 폴더 하위의 nlpbook/checkpoint-generation에 모델 체크포인트가 저장

- max_seq_length: 토큰 기준 입력 문장 최대 길이. 
  * 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰(PAD)을 붙여 줌

- batch_size: 배치 크기
  * 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 32, TPU라면(torch.cuda.is_available() == False) 4
  * 코랩 환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이기 때문에 이렇게 설정함

- learning_rate: 러닝레이트
  * 1회 스텝에서 한 번에 얼마나 업데이트할지에 관한 크기

- epochs: 학습 에폭 수
  * 3이라면 학습 데이터를 3회 반복 학습

- tpu_cores: TPU 코어 수 
  * 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 0, TPU라면(torch.cuda.is_available() == False) 8.

- seed: 랜덤 시드(정수, integer)
  * None이면 랜덤 시드를 고정하지 않음

In [3]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",  #40GB 이상의 텍스트로 학습된 한국어 디코더(decoder) 언어모델
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

- 랜덤 시드 고정

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


- 로거 설정

In [5]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cp

### 말뭉치 내려받기

- 실습에 사용할 말뭉치(KorQuAD 1.0)를 다운로드
- 코랩 환경 로컬의 root_dir(/content/Korpora)이하에 저장

In [6]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 317MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 153MB/s]


### 토크나이저 준비

- KoGPT2 모델(skt/kogpt2-base-v2)이 사용하는 토크나이저를 선언
- eos_token: 문장 마지막에 붙이는 스페셜 토큰, SK텔레콤이 모델을 프리트레인할 때 이렇게 지정했기 때문에 같은 방식으로 사용

In [7]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token='</s>',
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 데이터 전처리

#### 학습용 데이터 구축

- GenerationDataset: 각 instance를 포함하는 데이터셋 역할을 수행
  * 이 클래스는 NsmcCorpus와 토크나이저를 품고 있음 
  * NsmcCorpus가 넘겨준 문장을 input_ids, attention_mask, token_type_ids, labels 네 자료로 변환하는 역할

In [8]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 5.160 s]
INFO:ratsnlp:tokenize sentences [took 5.160 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> <

In [10]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

- input_ids: 극성 정보가 포함된 입력 문장을 토큰화한 뒤 이를 인덱싱한 결과
  * input_ids 뒤 쪽에 1이 많이 붙어 있음을 확인할 수 있음
  * 1은 토크나이저에서 정의한 eos_token(< / s >)에 대응하며 패딩 역할을 함

- attention_mask: 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타냄

- labels: input_ids와 같음
  * GPT2LMHeadModel이 파인튜닝 과정에서 labels를 오른족으로 한 칸씩 옮겨서 파인튜닝 태스크가 '입력 토큰의 다음 토큰 맞히기'가 되도록 함
  * 파인튜닝 과정: labels에 해당하는 확률은 높이고 이외의 토큰은 낮추는 방식으로 모델 전체를 업데이트 하는 것

In [9]:
train_dataset[0]

GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

#### 평가용 데이터 구축

In [11]:
from torch.utils.data import SequentialSampler
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='test',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 2.135 s]
INFO:ratsnlp:tokenize sentences [took 2.135 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


### 모델 초기화
- 프리트레인이 완료된 GPT2 모델을 읽고, 문장 생성을 수행할 모델을 초기화
- GPT2LMHeadModel: SK텔레콤이 공개한 KoGPT2가 프리트레인할 때 썼던 모델 클래스
- 프리트레인 태스크와 파인튜닝 태스크가 '다음 단어 맞히기'로 같기 때문에 똑같은 모델 클래스 사용


In [12]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name,
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

### 모델 학습

In [13]:
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [14]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
#폴더 복사
!cp -r /gdrive/MyDrive/CAKD8_Share/checkpoint-generation /gdrive/MyDrive/nlpbook/checkpoint-generation1